In [ ]:
'''
author: 梁泽涛，叶文涛
create time: 2020-6-27
update time: 2020-7-25
'''

In [1]:
import numpy as np
import pandas as pd

import pandas_profiling
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 用的是 date preprocess 中初筛数据 + B榜测试集
%time train_data = pd.read_pickle('data/clean_data1.pkl')
train_data['timestamp'] = pd.to_datetime(train_data['timestamp'])
# 去除没有路由的数据
train_data = train_data[~train_data['TRANSPORT_TRACE'].isnull()].reset_index(drop = True)

Wall time: 44.7 s


In [12]:
#构造港口的字典
port_data=pd.read_csv('data/port.csv')
port_data=port_data[['TRANS_NODE_NAME','LONGITUDE','LATITUDE']]

port_dict={}
for i in range(len(port_data)):
    port_dict[port_data['TRANS_NODE_NAME'].iloc[i]]=[port_data['LONGITUDE'].iloc[i],port_data['LATITUDE'].iloc[i]]

port_dict['BREMEN_GERMANY']=[8.75,53.116667]
port_dict['JEBEL ALI']=[55.0272904,24.9857145]
port_dict['MANZANILLO']=[-104.333333,19.05]
port_dict['MOMBASA']=[39.85,-3.466667]
port_dict['SHK']=[114.066667,22.516667]
port_dict['BUENOS AIRES']=[-58.366667,-34.6]
port_dict['HAKATA']=[130.45,33.616667]
port_dict['HONG KONG']=[114.25,22.3]
port_dict['INCHON']=[126.633333,37.55]
port_dict['KAMPALA WAREHOUSE_UGANDA']=[32.5825197,0.3475964]
port_dict['NZLYT']=[172.7,-43.583333]
port_dict['OSAKA']=[135.433333,34.65]
port_dict['RIYADH CITY_SAUDI ARABIA']=[46.694788,24.697075]
port_dict['SURABAYA']=[112.636986,-7.201896]
port_dict['YANGON WH_MYANMAR']=[96.190748,16.85484]
port_dict['NAIROBI']=[36.929846,-1.325971]
port_dict['PUERTO QUETZAL']=[-90.787534,13.940552]
port_dict['REYKJAVIK']=[-21.938826,64.149018]
port_dict['TRIPOLI']=[13.216667,32.9]
port_dict['VLADIVOSTOK']=[131.833333,43.166667]
port_dict['GUAYAQUIL']=[-79.90301,-2.261349]
port_dict['minsk']=[27.60217,53.750488]
port_dict['HONG KONG_HONG KONG']=[114.25,22.3]
port_dict['SAN JOSE']=[-90.833333,13.916667]
port_dict['BELAWAN']=[98.683333,3.783333]
port_dict['MYPKE']=[101.4086,3.005078]
port_dict['VNCLI']=[106.6296638,10.8230989]
port_dict['DURBAN']=[31.0218404,-29.8586804]
port_dict['BUENAVENTURA']=[-77.0197212,3.8830471]
port_dict['SINGAPORE']=[103.819836,1.352083]
port_dict["NUKU'ALOFA"]=[-175.204947,-21.1393418]
port_dict['MANAUS']=[-60.0217314,-3.1190275]
port_dict['LILONGWE']=[33.7741195,-13.9626121]
port_dict['LAHORE']=[74.3571581,31.5546061]
port_dict['KAMPALA']=[32.5825197,0.3475964]
port_dict['BRMAO']=[-60.0217314,-3.1190275]
port_dict['DAR ES SALAAM']=[39.2083284,-6.792354]
port_dict['KIEV']=[30.5234,50.4501]
port_dict['LUBUMBASHI']=[27.5026174,-11.6876026]
port_dict['GABORONE']=[25.9231471,-24.6282079]
port_dict['BFOUA']=[-1.5196603,12.3714277]
port_dict['NAWDH']=[17.0657549,-22.5608807]
port_dict['QINGDAO_CHINA']=[120.382609,36.067108]
port_dict['KRIBI']=[9.9101915,2.9405941]
port_dict['VALENCIA WAREHOUCE_SPAIN']=[-0.3762881,39.4699075]
port_dict['SOHAR']=[56.7075141,24.3460959]
port_dict['DAR ES SALAAM_TANZANIA']=[39.2083284,-6.792354]
port_dict['CAUCEDO']=[-69.6308326,18.425741]
port_dict['LAHORE']=[74.3571581,31.5546061]
port_dict['PORT SAID EAST']=[32.3018661,31.2652893]
port_dict['DOUALA']=[9.7678687,4.0510564]
port_dict['KINSHASA']=[15.2662931,-4.4419311]
port_dict['DRC']=[12.3893488,-5.9871641]
port_dict['BLANTYRE WAREHOUSE_MALAWI']=[35.0167866,-15.7666707]
port_dict['TOMAKOMAI']=[141.6053721,42.6343021]
port_dict['POINTE NOIRE_CONGO']=[11.866362,-4.7691623]
port_dict['CALLAO_PERU']=[-77.1259843,-12.0508491]
port_dict['ALEXANDRIA']=[29.9187387,31.2000924]
port_dict['ILASD']=[34.655314,31.804381]
port_dict['DOHA']=[51.5310398,25.2854473]
port_dict['MASERU']=[27.5143603,-29.3632188]
port_dict['JEDDAH']=[39.2375507,21.2854067]
port_dict['LAE']=[146.999905,-6.7155252]
port_dict['CHZRH']=[8.541694,47.3768866]
port_dict['BANGUI']=[18.5581899,4.3946735]
port_dict['BAKU']=[49.8670924,40.4092617]
port_dict['NIAMEY']=[2.1253854,13.5115963]
port_dict['MVMLI']=[73.5093474,4.1754959]

In [7]:
loadingOrderEvent_df = pd.read_csv('data/loadingOrderEvent.csv',nrows=158341)
test_data = pd.read_csv('data/testData 0626.csv')

In [49]:
# haversine 经纬度球面距离
def Distance2(LonA,LatA,LonB,LatB):
    EARTH_RADIUS = 6378.137 # 千米
    radLatA = np.radians(LatA)
    radLatB = np.radians(LatB)
    a = radLatA-radLatB
    b = np.radians(LonA)-np.radians(LonB)
    s= 2 * np.arcsin(np.sqrt(np.power(np.sin(a / 2),2)+ np.cos(radLatA) * np.cos(radLatB)*np.power(np.sin(b / 2),2))) * EARTH_RADIUS
    return s

def label_cut_v2(df,trace):
    #始末港口
    start_port = trace.split('-')[0]
    end_port = trace.split('-')[-1]
    
    #始末港口坐标
    start_port_coord = port_dict[start_port]
    end_port_coord = port_dict[end_port]
    
    #始末gps记录坐标
    first_record_coord = [df.longitude.values[0],df.latitude.values[0]]
    final_record_coord = [df.longitude.values[-1],df.latitude.values[-1]]
    
    # 初始记录到出发港口的距离
    dist2start =Distance2(start_port_coord[0],start_port_coord[1],first_record_coord[0],first_record_coord[1])
    if dist2start>50:
        return 0
    
    #默认出发时间
    start_time = df.timestamp.values[0]
    # 选取靠近起点港口的记录
    near_start_port_df = df[np.sqrt((df.longitude-start_port_coord[0])**2+ (df.latitude-start_port_coord[1])**2)<1]
    
    # 在靠近起点港口的记录中选取状态为moored的
    start_moored_df = near_start_port_df[near_start_port_df.vesselStatus=='moored']
    if len(start_moored_df)>0:
        start_time_idx = start_moored_df.index[-1]
        try:
            start_time = near_start_port_df.timestamp[start_time_idx+1]
        except:
            print('运单数据异常情况,keyerror')
            return 0
        
    # 选取靠近终点港口的记录
    near_end_port_df = df[np.sqrt((df.longitude-end_port_coord[0])**2+ (df.latitude-end_port_coord[1])**2)<0.7]
    # 如果没有靠近终点港口的记录则打标0
    if len(near_end_port_df)<=0:
        return 0
    
    min_speed_near_end = np.min(near_end_port_df.speed.values)
    end_time = near_end_port_df[near_end_port_df.speed==min_speed_near_end].timestamp.values[-1]
    # 在靠近终点港口的记录中选取状态为moored的
    end_moored_df = near_end_port_df[near_end_port_df.vesselStatus=='moored']
    if len(end_moored_df)>0:
        end_time_idx = end_moored_df.index[0]
        #print(near_end_port_df.index,end_time_idx-1)
        try:
            end_time = near_end_port_df.timestamp[end_time_idx-1]
        except:
            print('运单数据异常情况,keyerror')
            return 0
    
    # 针对到达港口后第一个moored停留数日的情况
    # 对于大于12小时时间间隔的记录，选择其上一个记录的时间，并与使用moored状态得出的时间比较，选择更早的时间。
    near_end_port_df.end_diff_hour = near_end_port_df.timestamp.diff(1).dt.total_seconds()/3600
    end_time_idx_by_diff_hour = near_end_port_df.index[(near_end_port_df.end_diff_hour>12)&
                                                        (near_end_port_df.end_diff_hour==near_end_port_df.end_diff_hour.max())]
    print(end_time_idx_by_diff_hour)
    if len(end_time_idx_by_diff_hour)>0:
        try:
            end_time = min(end_time, near_end_port_df.timestamp[end_time_idx_by_diff_hour[0]-1])
        except:
            print('运单数据异常情况,keyerror')
            return 0

        
    end_time = pd.to_datetime(end_time).tz_localize(None)
    start_time = pd.to_datetime(start_time).tz_localize(None)
    duration = (end_time-start_time).total_seconds()
    #diff_hour = df.timestamp.diff(1).dt.total_seconds()/3600
    return duration

In [48]:
tmp_route = 'CNSHK-MYTPP'
%time tmp_data =train_data[train_data['TRANSPORT_TRACE'].str.contains(tmp_route)].reset_index(drop=True)


Wall time: 18.1 s


In [ ]:
%time tmp_label = tmp_data.groupby(by='loadingOrder').apply(label_cut_v2,tmp_route)

In [45]:
# 查看打标情况
tmp_label

loadingOrder
AA191175561416    2177261.0
AU809825861449    2046873.0
AX121943322046    2162391.0
AZ165684400071    1910091.0
BC974719697835          0.0
                    ...    
ZH151511414153    2082909.0
ZO811542006703    2082909.0
ZQ157208081736          0.0
ZX741696557865    2067500.0
ZY989876486292    2165596.0
Length: 73, dtype: float64

In [16]:
tmp_label

loadingOrder
BY789350322657          0.0
GO496250630640    2360539.0
HU508667240863          0.0
HZ571673766531          0.0
IC869966733240          0.0
KU436448486469    2191390.0
NZ438363836923    2454392.0
QZ477152068239    2528349.0
SM614397332616    2058393.0
ZM490479048723    2485415.0
dtype: float64

In [19]:
tmp_label

loadingOrder
BY789350322657          0.0
GO496250630640    2360539.0
HU508667240863          0.0
HZ571673766531          0.0
IC869966733240          0.0
KU436448486469    2191390.0
NZ438363836923    2454392.0
QZ477152068239    2528349.0
SM614397332616    2058393.0
ZM490479048723    2485415.0
dtype: float64